## Линейная регрессия: прогноз оклада по описанию вакансии

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge

from scipy.sparse import hstack

import re

In [2]:
def write_answer(filename, answer):
    
    with open(filename, 'w') as fout:
        
        fout.write(str(answer))
        fout.close()

### Введение

Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

### Реализация в Scikit-Learn

Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом <code>sklearn.feature_extraction.text.TfidfVectorizer</code>. 

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе <code>sklearn.linear_model.Ridge</code>.

Обратите внимание, что признаки LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки LocationNormalized и ContractTime. Он уже реализован в классе <code>sklearn.feature_extraction.DictVectorizer</code>. Пример использования:
```python
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))
```
Вам понадобится производить замену пропущенных значений на специальные строковые величины (например, 'nan'). Для этого подходит следующий код:
```python
data_train['LocationNormalized'].fillna('nan', inplace=True)
data_train['ContractTime'].fillna('nan', inplace=True)

```

####  1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv (либо его заархивированную версию salary-train.zip).

In [3]:
train = pd.read_csv('salary-train.csv')
train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


#### 2.     Проведите предобработку:

- Приведите тексты к нижнему регистру (<code>text.lower()</code>).
- Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: <code>re.sub('[^a-zA-Z0-9]', ' ', text)</code>. Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:
```python
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)
```
- Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).
- Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.
- Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.
- Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

1. Приведем текст к нижнему регистру

In [4]:
train['FullDescription'] = train['FullDescription'].str.lower()
train['LocationNormalized'] = train['LocationNormalized'].str.lower()
train['ContractTime'] = train['ContractTime'].str.lower()

train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london ****k ****...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager// luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse / rgn nursing home for young...,sutton coldfield,NaN,20355


2. Заменим в тексте все, кроме букв и цифр, на пробелы

In [5]:
train['FullDescription'] = train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex=True)

train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,london,permanent,33000
1,an ideal opportunity for an individual that ha...,london,permanent,50000
2,online content and brand manager luxury reta...,south east london,permanent,40000
3,a great local marketleader is seeking a perman...,dereham,permanent,22500
4,registered nurse rgn nursing home for young...,sutton coldfield,NaN,20355


3. Применим TfidfVectorizer к текстам (min_df=5)

In [6]:
vectorizer = TfidfVectorizer(min_df=5)

X_text_vec = vectorizer.fit_transform(train['FullDescription'])

4. Заменим пропуски в столбцах LocationNormalized и ContractTime на строку 'nan'

In [7]:
train['LocationNormalized'].fillna('nan', inplace=True)
train['ContractTime'].fillna('nan', inplace=True)

5. Применим DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime

In [8]:
one_hot = DictVectorizer()

X_train_cat = one_hot.fit_transform(train[['LocationNormalized', 'ContractTime']].to_dict('records'))

6. Объединим все полученные признаки в одну матрицу "объекты-признаки"

In [9]:
X_train = hstack((X_text_vec, X_train_cat))
X_train

<60000x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 8485759 stored elements in COOrdinate format>

In [10]:
y_train = train['SalaryNormalized']

#### 3.  Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [11]:
ridge_reg = Ridge(alpha=1, random_state=241)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

#### 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv. Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.42. При необходимости округляйте дробную часть до двух знаков.

In [12]:
test = pd.read_csv('salary-test-mini.csv')
test

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


Проведем предобработку тестовых данных

In [13]:
test['FullDescription'] = test['FullDescription'].str.lower()
test['LocationNormalized'] = test['LocationNormalized'].str.lower()
test['ContractTime'] = test['ContractTime'].str.lower()

test['LocationNormalized'].fillna('nan', inplace=True)
test['ContractTime'].fillna('nan', inplace=True)

test['FullDescription'] = test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex=True)

X_test_vec = vectorizer.transform(test['FullDescription'])

X_test_cat = one_hot.transform(test[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_test = hstack((X_test_vec, X_test_cat))
X_test

<2x24627 sparse matrix of type '<class 'numpy.float64'>'
	with 304 stored elements in COOrdinate format>

In [14]:
predictions = ridge_reg.predict(X_test)
predictions

array([56567.99943785, 37143.80024961])

In [15]:
write_answer('submission_ridge_reg_1.txt', ' '.join(list(map(str, map(lambda x: round(x, 2), list(map(float, predictions)))))))